# etlFFIEC Database Walkabout
There are four tables in the database. They are `dictionary`, `institution`, `period`, and `report`.

Each is detailed below. At the bottom, there's an example of analysis with pandas and SQL.

In [476]:
import json
from datetime import datetime
from pprint import pprint

import happybase
import pandas
import pandasql
import matplotlib

HBASE = 'node1.hwr.io'  # CHANGE ME
SAMPLE_BANKS = (131034, 720858, 229342, 819172, 65513, 753641)

def stringify_dict(dictionary):
    # Because everything in Hbase is a byte-array,
    # we need to convert the keys/values of retrieved dictionaries into unicode strings
    
    # also, mildly annoying, but the dict keys retain the Hbase column name, pop that off
    return {key.decode('utf-8').split(':', 1).pop(-1): 
            value.decode('utf-8') for key,value in dictionary.items()}

def period_to_datetime(period):
    return datetime.strptime(period, '%m/%d/%Y')

def datetime_to_period(value):
    return datetime.strftime(value, '%m/%d/%Y')

def filter_periods(periods, count, reverse):
    # convert string datestamps to python datetime, sort, return a slice of `count` length
    periods_we_care_about = [period_to_datetime(k) for k in periods]
    periods_we_care_about.sort(reverse=reverse)
    return[datetime_to_period(k) for k in periods_we_care_about[:count]]

# The `dictionary` table

`dictionary` has column `M`, for Metadata. The row-key format for `dictionary` is the MDRM mnemonic and item code appended together, making the MDRM Identifier. It's eight all-uppercase alphanumeric characters, and looks something like this `RCON2170`.

This is the only table not populated from the FFIEC CDR. Instead, it is loaded from a CSV file downloaded from the Federal Reserve. Since this CSV essentially becomes a dependancy of the project, it is dowloaded into the docker image at build time. This table is populated in two scenarios, either when the `--init` flag is passed or when the `--update-metadata` flag is passed. 

In [477]:
# Scan the entire dictionary table into memory
hbase = happybase.Connection(HBASE)
dictionary = hbase.table('dictionary')
items = [item for item in dictionary.scan()]

mdrm = {}
for row, document in items:
    mdrm[row.decode('utf-8')] = stringify_dict(document)

In [478]:
for k,v in mdrm.items():
    pprint(k)
    pprint(v)
    break

'SUBC3545'
{'confidentiality': 'N',
 'description': 'Banks that (a) regularly underwrite or deal in securities; '
                'interest rate contracts; foreign exchange rate contracts; '
                'other off-balance sheet commodity and equity contracts; other '
                'financial instruments; and other assets for resale; (b) '
                'acquire or take positions in such items principally for the '
                'purpose of selling in the near term or otherwise with the '
                'intent to resell in order to profit from short-term price '
                'movements; or (c) acquire or take positions in such items as '
                'an accommodation to customers or for other trading purposes '
                'shall report in this item the value of such assets or '
                'positions on the report date. Assets and other financial '
                'instruments held for trading shall be consistently valued at '
                'fair value (or;

# The `period` table

`period` has column `I`, for institution. The row-key for `period` is the datestamp format used by the FFIEC to represent a reporting period. This table is used for looking up all institutions which have filed call reports for a given reporting period. Each column value is a json-encoded document of metadata about the institution at the given period.

In [479]:
hbase = happybase.Connection(HBASE)
period_table = hbase.table('period')

periods = {}
for row, value in period_table.scan():
    document = stringify_dict(value)
    for key in document:
        document[key] = json.loads(document[key])
    
    periods[row.decode('utf-8')] = document


pprint(periods)

{'12/31/2001': {'131034': {'Address': '156 GENEVA DRIVE',
                           'City': 'OVIEDO',
                           'FDICCertNumber': 16442,
                           'FilingType': '041',
                           'HasFiledForReportingPeriod': True,
                           'ID_RSSD': 131034,
                           'Name': 'CITIZENS BANK OF OVIEDO, THE',
                           'OCCChartNumber': 0,
                           'OTSDockNumber': 0,
                           'PrimaryABARoutNumber': 63105544,
                           'State': 'FL',
                           'ZIP': 32765},
                '229342': {'Address': '1025 6TH STREET',
                           'City': 'NEVADA',
                           'FDICCertNumber': 15651,
                           'FilingType': '041',
                           'HasFiledForReportingPeriod': True,
                           'ID_RSSD': 229342,
                           'Name': 'STATE BANK & TRUST CO.',
         

                '819172': {'Address': '10 MONUMENT SQUARE',
                           'City': 'URBANA',
                           'FDICCertNumber': 29975,
                           'FilingType': '041',
                           'HasFiledForReportingPeriod': True,
                           'ID_RSSD': 819172,
                           'Name': 'PEOPLES SAVINGS BANK, THE',
                           'OCCChartNumber': 0,
                           'OTSDockNumber': 4468,
                           'PrimaryABARoutNumber': 244171902,
                           'State': 'OH',
                           'ZIP': 43078}},
 '3/31/2001': {'131034': {'Address': '156 GENEVA DRIVE',
                          'City': 'OVIEDO',
                          'FDICCertNumber': 16442,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 131034,
                          'Name': 'CITIZENS BANK OF OVIEDO, THE',
     

               '753641': {'Address': '200 EAST ERIE STREET',
                          'City': 'ROGERS CITY',
                          'FDICCertNumber': 23126,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 753641,
                          'Name': 'HURON NATIONAL BANK',
                          'OCCChartNumber': 16857,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 72413285,
                          'State': 'MI',
                          'ZIP': 49779},
               '819172': {'Address': '10 MONUMENT SQUARE',
                          'City': 'URBANA',
                          'FDICCertNumber': 29975,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 819172,
                          'Name': 'PEOPLES SAVINGS BANK, THE',
                  

                          'ZIP': 50201},
               '65513': {'Address': '650 FIFTH AVE 26 FL',
                         'City': 'NEW YORK',
                         'FDICCertNumber': 26633,
                         'FilingType': '031',
                         'HasFiledForReportingPeriod': True,
                         'ID_RSSD': 65513,
                         'Name': 'DELTA NATIONAL BANK AND TRUST COMPANY',
                         'OCCChartNumber': 20547,
                         'OTSDockNumber': 0,
                         'PrimaryABARoutNumber': 26010773,
                         'State': 'NY',
                         'ZIP': 10019},
               '720858': {'Address': '204 WEST MAIN ST',
                          'City': 'ASHLEY',
                          'FDICCertNumber': 13794,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 720858,
                          'Name': 'MCINTO

                          'State': 'OH',
                          'ZIP': 43078}},
 '9/30/2008': {'131034': {'Address': '156 GENEVA DRIVE',
                          'City': 'OVIEDO',
                          'FDICCertNumber': 16442,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 131034,
                          'Name': 'CITIZENS BANK OF FLORIDA',
                          'OCCChartNumber': 0,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 63105544,
                          'State': 'FL',
                          'ZIP': 32765},
               '229342': {'Address': '1025 6TH STREET',
                          'City': 'NEVADA',
                          'FDICCertNumber': 15651,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 229342,
         

# The `institution` table
`institution` has column `P`, for period. The row-key format is an [RSSDID](https://www.alacra.com/alacra/outside/lei/info/rssdid.html) which stands for 'Replication Server System Database ID', got all that? This table is used for looking up all reporting periods in which you'll find call reports for the given institution. 


In [480]:
hbase = happybase.Connection(HBASE)
institution_table = hbase.table('institution')

institutions = {}
for row, value in institution_table.scan():
    document = stringify_dict(value)
    for key in document:
        document[key] = json.loads(document[key])
    
    institutions[row.decode('utf-8')] = document
    
pprint(institutions)

{'131034': {'12/31/2001': {'Address': '156 GENEVA DRIVE',
                           'City': 'OVIEDO',
                           'FDICCertNumber': 16442,
                           'FilingType': '041',
                           'HasFiledForReportingPeriod': True,
                           'ID_RSSD': 131034,
                           'Name': 'CITIZENS BANK OF OVIEDO, THE',
                           'OCCChartNumber': 0,
                           'OTSDockNumber': 0,
                           'PrimaryABARoutNumber': 63105544,
                           'State': 'FL',
                           'ZIP': 32765},
            '12/31/2002': {'Address': '156 GENEVA DRIVE',
                           'City': 'OVIEDO',
                           'FDICCertNumber': 16442,
                           'FilingType': '041',
                           'HasFiledForReportingPeriod': True,
                           'ID_RSSD': 131034,
                           'Name': 'CITIZENS BANK OF OVIEDO, THE',
  

                          'Name': 'STATE BANK & TRUST CO.',
                          'OCCChartNumber': 0,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 73921831,
                          'State': 'IA',
                          'ZIP': 50201},
            '3/31/2011': {'Address': '1025 6TH STREET',
                          'City': 'NEVADA',
                          'FDICCertNumber': 15651,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 229342,
                          'Name': 'STATE BANK & TRUST CO.',
                          'OCCChartNumber': 0,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 73921831,
                          'State': 'IA',
                          'ZIP': 50201},
            '3/31/2012': {'Address': '1025 6TH STREET',
                          'City': 'NEVADA',
     

                         'Name': 'DELTA NATIONAL BANK AND TRUST COMPANY',
                         'OCCChartNumber': 20547,
                         'OTSDockNumber': 0,
                         'PrimaryABARoutNumber': 26010773,
                         'State': 'NY',
                         'ZIP': 10019},
           '9/30/2006': {'Address': '650 FIFTH AVE 26 FL',
                         'City': 'NEW YORK',
                         'FDICCertNumber': 26633,
                         'FilingType': '031',
                         'HasFiledForReportingPeriod': True,
                         'ID_RSSD': 65513,
                         'Name': 'DELTA NATIONAL BANK AND TRUST COMPANY',
                         'OCCChartNumber': 20547,
                         'OTSDockNumber': 0,
                         'PrimaryABARoutNumber': 26010773,
                         'State': 'NY',
                         'ZIP': 10019},
           '9/30/2007': {'Address': '650 FIFTH AVE 26 FL',
                     

                          'Name': 'HURON NATIONAL BANK',
                          'OCCChartNumber': 16857,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 72413285,
                          'State': 'MI',
                          'ZIP': 49779},
            '3/31/2002': {'Address': '200 EAST ERIE STREET',
                          'City': 'ROGERS CITY',
                          'FDICCertNumber': 23126,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 753641,
                          'Name': 'HURON NATIONAL BANK',
                          'OCCChartNumber': 16857,
                          'OTSDockNumber': 0,
                          'PrimaryABARoutNumber': 72413285,
                          'State': 'MI',
                          'ZIP': 49779},
            '3/31/2003': {'Address': '200 EAST ERIE STREET',
                          'City'

                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 819172,
                          'Name': 'PEOPLES SAVINGS BANK, THE',
                          'OCCChartNumber': 0,
                          'OTSDockNumber': 4468,
                          'PrimaryABARoutNumber': 244171902,
                          'State': 'OH',
                          'ZIP': 43078},
            '3/31/2012': {'Address': '10 MONUMENT SQUARE',
                          'City': 'URBANA',
                          'FDICCertNumber': 29975,
                          'FilingType': '041',
                          'HasFiledForReportingPeriod': True,
                          'ID_RSSD': 819172,
                          'Name': 'PEOPLES SAVINGS BANK, THE',
                          'OCCChartNumber': 0,
                          'OTSDockNumber': 4468,
                          'PrimaryABARoutNumber': 244171902,
                          'State': 'OH',
                      

In [481]:
hbase = happybase.Connection('node1.hwr.io', timeout=300000)  # connection with 5-minute timeout
reports_table = hbase.table('report')

# One usability improvement would be to reverse the order of the report period datastamp,
# which would allow for scanning institution by year, like `row_prefix=b'534242-2017'`

# having period formatted '%Y-%m-%d' would suit sqlite's date function as well

# It may also be better to create a document for each reported metric, 
# in the same fashion as the `institution` and `period` tables,
# rather than N column entries for N fields

# Regardless, this is the time to go get coffee
call_reports = {}
for bank in SAMPLE_BANKS:
    prefix = bytes('{}-'.format(bank), 'utf-8')
    for row, report in reports_table.scan(row_prefix=prefix):
        call_reports[row.decode('utf-8')] = stringify_dict(report)

In [482]:
for k,v in call_reports.items():
    pprint(k)
    pprint(v)
    break

'65513-3/31/2014'
{'RCFA3792:bank_rssd_identifier': '65513',
 'RCFA3792:call_date': '20140331',
 'RCFA3792:call_schedule': 'RCRIB',
 'RCFA3792:last_update': '20140429',
 'RCFA3792:line_number': '35a',
 'RCFA3792:mdrm_#': 'RCFA3792',
 'RCFA3792:short_definition': 'Total capital (sum of items 26 and 34.a)',
 'RCFA3792:value': '',
 'RCFA5310:bank_rssd_identifier': '65513',
 'RCFA5310:call_date': '20140331',
 'RCFA5310:call_schedule': 'RCRIB',
 'RCFA5310:last_update': '20140429',
 'RCFA5310:line_number': '30a',
 'RCFA5310:mdrm_#': 'RCFA5310',
 'RCFA5310:short_definition': 'Allowance for loan and lease losses includable '
                              'in tier 2 capital',
 'RCFA5310:value': '',
 'RCFA5311:bank_rssd_identifier': '65513',
 'RCFA5311:call_date': '20140331',
 'RCFA5311:call_schedule': 'RCRIB',
 'RCFA5311:last_update': '20140429',
 'RCFA5311:line_number': '34a',
 'RCFA5311:mdrm_#': 'RCFA5311',
 'RCFA5311:short_definition': 'Tier 2 capital (greater of item 32.a minus item '
     

 'RCFD1594:value': '0',
 'RCFD1597:bank_rssd_identifier': '65513',
 'RCFD1597:call_date': '20140331',
 'RCFD1597:call_schedule': 'RCN',
 'RCFD1597:last_update': '20140429',
 'RCFD1597:line_number': '3',
 'RCFD1597:mdrm_#': 'RCFD1597',
 'RCFD1597:short_definition': 'Loans to finance agricultural production and '
                              'other loans to farmers',
 'RCFD1597:value': '0',
 'RCFD1651:bank_rssd_identifier': '65513',
 'RCFD1651:call_date': '20140331',
 'RCFD1651:call_schedule': 'RCRII',
 'RCFD1651:last_update': '20140429',
 'RCFD1651:line_number': '58',
 'RCFD1651:mdrm_#': 'RCFD1651',
 'RCFD1651:short_definition': 'Market risk equivalent assets',
 'RCFD1651:value': '0',
 'RCFD1737:bank_rssd_identifier': '65513',
 'RCFD1737:call_date': '20140331',
 'RCFD1737:call_schedule': 'RCB',
 'RCFD1737:last_update': '20140429',
 'RCFD1737:line_number': '6a',
 'RCFD1737:mdrm_#': 'RCFD1737',
 'RCFD1737:short_definition': 'Other domestic debt securities',
 'RCFD1737:value': '52532',
 '

 'RCFD5378:short_definition': 'To U.S. banks and other U.S. depository '
                              'institutions',
 'RCFD5378:value': '0',
 'RCFD5379:bank_rssd_identifier': '65513',
 'RCFD5379:call_date': '20140331',
 'RCFD5379:call_schedule': 'RCN',
 'RCFD5379:last_update': '20140429',
 'RCFD5379:line_number': '2a',
 'RCFD5379:mdrm_#': 'RCFD5379',
 'RCFD5379:short_definition': 'To U.S. banks and other U.S. depository '
                              'institutions',
 'RCFD5379:value': '0',
 'RCFD5380:bank_rssd_identifier': '65513',
 'RCFD5380:call_date': '20140331',
 'RCFD5380:call_schedule': 'RCN',
 'RCFD5380:last_update': '20140429',
 'RCFD5380:line_number': '2b',
 'RCFD5380:mdrm_#': 'RCFD5380',
 'RCFD5380:short_definition': 'To foreign banks',
 'RCFD5380:value': '0',
 'RCFD5381:bank_rssd_identifier': '65513',
 'RCFD5381:call_date': '20140331',
 'RCFD5381:call_schedule': 'RCN',
 'RCFD5381:last_update': '20140429',
 'RCFD5381:line_number': '2b',
 'RCFD5381:mdrm_#': 'RCFD5381',
 'RC

 'RCFDA519:value': '0',
 'RCFDA520:bank_rssd_identifier': '65513',
 'RCFDA520:call_date': '20140331',
 'RCFDA520:call_schedule': 'RCF',
 'RCFDA520:last_update': '20140429',
 'RCFDA520:line_number': '3b',
 'RCFDA520:mdrm_#': 'RCFDA520',
 'RCFDA520:short_definition': 'Other financial assets',
 'RCFDA520:value': '0',
 'RCFDA549:bank_rssd_identifier': '65513',
 'RCFDA549:call_date': '20140331',
 'RCFDA549:call_schedule': 'RCB',
 'RCFDA549:last_update': '20140429',
 'RCFDA549:line_number': 'M2a1',
 'RCFDA549:mdrm_#': 'RCFDA549',
 'RCFDA549:short_definition': 'Three months or less',
 'RCFDA549:value': '8432',
 'RCFDA550:bank_rssd_identifier': '65513',
 'RCFDA550:call_date': '20140331',
 'RCFDA550:call_schedule': 'RCB',
 'RCFDA550:last_update': '20140429',
 'RCFDA550:line_number': 'M2a2',
 'RCFDA550:mdrm_#': 'RCFDA550',
 'RCFDA550:short_definition': 'Over three months through 12 months',
 'RCFDA550:value': '9705',
 'RCFDA551:bank_rssd_identifier': '65513',
 'RCFDA551:call_date': '20140331',
 

 'RCFDB668:call_schedule': 'RCRII',
 'RCFDB668:last_update': '20140429',
 'RCFDB668:line_number': '48',
 'RCFDB668:mdrm_#': 'RCFDB668',
 'RCFDB668:short_definition': 'Securities lent',
 'RCFDB668:value': '0',
 'RCFDB669:bank_rssd_identifier': '65513',
 'RCFDB669:call_date': '20140331',
 'RCFDB669:call_schedule': 'RCRII',
 'RCFDB669:last_update': '20140429',
 'RCFDB669:line_number': '49',
 'RCFDB669:mdrm_#': 'RCFDB669',
 'RCFDB669:short_definition': 'Retained recourse on small business obligations '
                              'sold with recourse',
 'RCFDB669:value': '0',
 'RCFDB670:bank_rssd_identifier': '65513',
 'RCFDB670:call_date': '20140331',
 'RCFDB670:call_schedule': 'RCRII',
 'RCFDB670:last_update': '20140429',
 'RCFDB670:line_number': '49',
 'RCFDB670:mdrm_#': 'RCFDB670',
 'RCFDB670:short_definition': 'Retained recourse on small business obligations '
                              'sold with recourse',
 'RCFDB670:value': '0',
 'RCFDB671:bank_rssd_identifier': '65513',
 'RCFD

...

 'RCON5569:call_schedule': 'RCCII',
 'RCON5569:last_update': '20140429',
 'RCON5569:line_number': '3c',
 'RCON5569:mdrm_#': 'RCON5569',
 'RCON5569:short_definition': 'With original amounts of more than $250,000 '
                              'through $1,000,000',
 'RCON5569:value': '0',
 'RCON5570:bank_rssd_identifier': '65513',
 'RCON5570:call_date': '20140331',
 'RCON5570:call_schedule': 'RCCII',
 'RCON5570:last_update': '20140429',
 'RCON5570:line_number': '4a',
 'RCON5570:mdrm_#': 'RCON5570',
 'RCON5570:short_definition': 'With original amounts of $100,000 or less',
 'RCON5570:value': '1',
 'RCON5571:bank_rssd_identifier': '65513',
 'RCON5571:call_date': '20140331',
 'RCON5571:call_schedule': 'RCCII',
 'RCON5571:last_update': '20140429',
 'RCON5571:line_number': '4a',
 'RCON5571:mdrm_#': 'RCON5571',
 'RCON5571:short_definition': 'With original amounts of $100,000 or less',
 'RCON5571:value': '61',
 'RCON5572:bank_rssd_identifier': '65513',
 'RCON5572:call_date': '20140331',
 'RCON

 'RCONF160:line_number': '1e1',
 'RCONF160:mdrm_#': 'RCONF160',
 'RCONF160:short_definition': 'Loans secured by owner-occupied nonfarm '
                              'nonresidential properties',
 'RCONF160:value': '0',
 'RCONF161:bank_rssd_identifier': '65513',
 'RCONF161:call_date': '20140331',
 'RCONF161:call_schedule': 'RCCI',
 'RCONF161:last_update': '20140429',
 'RCONF161:line_number': '1e2',
 'RCONF161:mdrm_#': 'RCONF161',
 'RCONF161:short_definition': 'Loans secured by other nonfarm nonresidential '
                              'properties',
 'RCONF161:value': '0',
 'RCONF172:bank_rssd_identifier': '65513',
 'RCONF172:call_date': '20140331',
 'RCONF172:call_schedule': 'RCN',
 'RCONF172:last_update': '20140429',
 'RCONF172:line_number': '1a1',
 'RCONF172:mdrm_#': 'RCONF172',
 'RCONF172:short_definition': '1-4 family residential construction loans',
 'RCONF172:value': '0',
 'RCONF173:bank_rssd_identifier': '65513',
 'RCONF173:call_date': '20140331',
 'RCONF173:call_schedule': 'R

                              'properties',
 'RCONK071:value': '0',
 'RCONK105:bank_rssd_identifier': '65513',
 'RCONK105:call_date': '20140331',
 'RCONK105:call_schedule': 'RCN',
 'RCONK105:last_update': '20140429',
 'RCONK105:line_number': 'M1a1',
 'RCONK105:mdrm_#': 'RCONK105',
 'RCONK105:short_definition': '1-4 family residential construction loans',
 'RCONK105:value': '0',
 'RCONK106:bank_rssd_identifier': '65513',
 'RCONK106:call_date': '20140331',
 'RCONK106:call_schedule': 'RCN',
 'RCONK106:last_update': '20140429',
 'RCONK106:line_number': 'M1a1',
 'RCONK106:mdrm_#': 'RCONK106',
 'RCONK106:short_definition': '1-4 family residential construction loans',
 'RCONK106:value': '0',
 'RCONK107:bank_rssd_identifier': '65513',
 'RCONK107:call_date': '20140331',
 'RCONK107:call_schedule': 'RCN',
 'RCONK107:last_update': '20140429',
 'RCONK107:line_number': 'M1a1',
 'RCONK107:mdrm_#': 'RCONK107',
 'RCONK107:short_definition': '1-4 family residential construction loans',
 'RCONK107:value'

 'RIAD4488:mdrm_#': 'RIAD4488',
 'RIAD4488:short_definition': 'Applicable income tax effect',
 'RIAD4488:value': '0',
 'RIAD4489:bank_rssd_identifier': '65513',
 'RIAD4489:call_date': '20140331',
 'RIAD4489:call_schedule': 'RIE',
 'RIAD4489:last_update': '20140429',
 'RIAD4489:line_number': '3c2',
 'RIAD4489:mdrm_#': 'RIAD4489',
 'RIAD4489:short_definition': 'Amount of component',
 'RIAD4489:value': '0',
 'RIAD4491:bank_rssd_identifier': '65513',
 'RIAD4491:call_date': '20140331',
 'RIAD4491:call_schedule': 'RIE',
 'RIAD4491:last_update': '20140429',
 'RIAD4491:line_number': '3c3',
 'RIAD4491:mdrm_#': 'RIAD4491',
 'RIAD4491:short_definition': 'Applicable income tax effect',
 'RIAD4491:value': '0',
 'RIAD4498:bank_rssd_identifier': '65513',
 'RIAD4498:call_date': '20140331',
 'RIAD4498:call_schedule': 'RIE',
 'RIAD4498:last_update': '20140429',
 'RIAD4498:line_number': '5a2',
 'RIAD4498:mdrm_#': 'RIAD4498',
 'RIAD4498:short_definition': 'Amount of component',
 'RIAD4498:value': '0',
 'R

 'RIADF555:mdrm_#': 'RIADF555',
 'RIADF555:short_definition': 'Bank card and credit card interchange fees',
 'RIADF555:value': '0',
 'RIADF556:bank_rssd_identifier': '65513',
 'RIADF556:call_date': '20140331',
 'RIADF556:call_schedule': 'RIE',
 'RIADF556:last_update': '20140429',
 'RIADF556:line_number': '2h',
 'RIADF556:mdrm_#': 'RIADF556',
 'RIADF556:short_definition': 'Accounting and auditing expenses',
 'RIADF556:value': '71',
 'RIADF557:bank_rssd_identifier': '65513',
 'RIADF557:call_date': '20140331',
 'RIADF557:call_schedule': 'RIE',
 'RIADF557:last_update': '20140429',
 'RIADF557:line_number': '2i',
 'RIADF557:mdrm_#': 'RIADF557',
 'RIADF557:short_definition': 'Consulting and advisory expenses',
 'RIADF557:value': '0',
 'RIADF558:bank_rssd_identifier': '65513',
 'RIADF558:call_date': '20140331',
 'RIADF558:call_schedule': 'RIE',
 'RIADF558:last_update': '20140429',
 'RIADF558:line_number': '2j',
 'RIADF558:mdrm_#': 'RIADF558',
 'RIADF558:short_definition': 'Automated teller mac

In [483]:
# dig for interesting asset-related reports, enrich the item_name

possible_relevent_fields = {}

for key, value in mdrm.items():
    item_type = value['item_type']
    item_name = value['item_name'].lower()
    if 'total asset' in item_name:   
        possible_relevent_fields[key] = '{} {}'.format(item_type, item_name)

pprint(possible_relevent_fields)

{'AAAA2170': 'reported total assets',
 'AMLB2180': 'reported total assets; less cash and due from related and '
             'unrelated entities',
 'AMLW2180': 'reported total assets; less cash and due from related and '
             'unrelated entities',
 'BAAQ3368': 'derived quarterly average of total assets',
 'BAAY3368': 'derived quarterly average of total assets',
 'BAGD2170': 'derived total assets',
 'BAGQL254': 'derived trading account income to total assets percent',
 'BAGR2170': 'derived total assets',
 'BAGR2192': 'derived total assets (held in domestic offices)',
 'BAGR5320': 'derived total assets (0% risk-weight)',
 'BAGR5327': 'derived total assets (20% risk-weight)',
 'BAGR5334': 'derived total assets (50% risk-weight)',
 'BAGR5340': 'derived total assets (100% risk-weight)',
 'BAGRA224': 'derived average total assets (net of deductions)',
 'BAGRB505': 'derived adjustment to average total assets reported in item no. '
             'a224',
 'BAGRB644': 'derived total asset

In [485]:
# process call_reports into a time-series
data = []
for row_key, call_report in call_reports.items():  
    rssd, period = row_key.split('-')
    
    for key, value in call_report.items():
        mdrm_id, name = key.split(':')
        
        if mdrm_id not in possible_relevent_fields:
            continue
        
        if name != 'value':
            continue
                
        try:
            value = float(value)
            if value.is_integer():
                value = int(value)
        except ValueError:
            pass

        if isinstance(value, str) and not value:
            value = None
            
        data.append({'institution': institutions[rssd][period]['Name'],
                     'mdrm': mdrm_id,
                     'type': mdrm[mdrm_id]['item_type'],
                     'name': mdrm[mdrm_id]['item_name'],
                     'rssd': int(rssd), 
                     'value': value,
                     'period': period_to_datetime(period)})

for item in data:
    pprint(item)
    break

{'institution': 'DELTA NATIONAL BANK AND TRUST COMPANY',
 'mdrm': 'RCFDL138',
 'name': 'TOTAL ASSETS FOR LEVERAGE CAPITAL PURPOSES',
 'period': datetime.datetime(2014, 3, 31, 0, 0),
 'rssd': 65513,
 'type': 'reported',
 'value': 452034}



# Ok, so can you answer the question?
> how has asset growth over time compared with other / similar banks?

We'll look at MDRM ID RCON2170, derrived total assets.

In [467]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
reports_frame = pandas.DataFrame(data=data)

In [468]:
# let's compare 131034, 819172, 720858, 229342, 819172, 753641
# it seems there are no RCON2170 values for 65513
pysqldf('''select rssd, institution, type, name, strftime('%Y', period) as year, sum(value)
                  from reports_frame where mdrm="RCON2170"
                  GROUP BY year, rssd
                  ORDER BY rssd, year''')

,rssd,institution,type,name,year,sum(value)
0,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2001,437415.0
1,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2002,493609.0
2,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2003,536692.0
3,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2004,584885.0
4,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2005,663896.0
5,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2006,731743.0
6,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2007,751258.0
7,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2008,961270.0
8,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2009,915913.0
9,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2010,914478.0


In [469]:
pysqldf('''select institution, type, name, mdrm, rssd, strftime('%Y', period) as year, sum(value)
                  from reports_frame where mdrm="RCFD5320" 
                  GROUP BY year, rssd
                  ORDER BY year, rssd''')

,institution,type,name,mdrm,rssd,year,sum(value)
0,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2001,488178.0
1,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2002,458185.0
2,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2003,383452.0
3,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2004,255093.0
4,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2005,182524.0
5,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2006,113664.0
6,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2007,85282.0
7,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2008,318283.0
8,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2009,927706.0
9,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2010,935286.0


In [486]:
pysqldf('''select institution, type, name, mdrm, rssd, strftime('%Y', period) as year, sum(value)
                  from reports_frame
                  GROUP BY year, rssd
                  ORDER BY year, rssd, mdrm''')

,institution,type,name,mdrm,rssd,year,sum(value)
0,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCFDB698,65513,2001,10148685.0
1,"CITIZENS BANK OF OVIEDO, THE",reported,TOTAL ASSETS (0% RISK-WEIGHT),RCON5320,131034,2001,2188046.0
2,STATE BANK & TRUST CO.,reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCONB698,229342,2001,1991273.0
3,MCINTOSH COUNTY BANK,reported,TOTAL ASSETS (20% RISK-WEIGHT),RCON5327,720858,2001,1102373.0
4,HURON NATIONAL BANK,reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCONB698,753641,2001,850717.0
5,"PEOPLES SAVINGS BANK, THE",reported,TOTAL ASSETS (0% RISK-WEIGHT),RCON5320,819172,2001,1467561.0
6,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2002,9939401.0
7,"CITIZENS BANK OF OVIEDO, THE",reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCONB698,131034,2002,2470164.0
8,STATE BANK & TRUST CO.,reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCONB698,229342,2002,2052548.0
9,MCINTOSH COUNTY BANK,reported,TOTAL ASSETS; DERIVATIVES; AND OFF-BALANCE SHE...,RCONB698,720858,2002,1120218.0
